### ANÁLISIS DE VALIDACIÓN Y OPTIMIZACIÓN DEL AGENTE

- Juan Miguel Dimate
- Andrea Forero
- Juan David Sánchez


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
from connect4.policy import Policy
from connect4.utils import find_importable_classes
from connect4.connect_state import ConnectState
from tournament import run_tournament, play

In [ ]:
participants = find_importable_classes("groups", Policy)

players = list(participants.items())

player_names = []

for name, policy in players:
    player_names.append(name)
    
num_agents = len(player_names)

In [ ]:

num_runs = 10            # Número de veces que se ejecuta el torneo completo
seed = 911

## Competencia eliminatoria

In [ ]:
champion_count = {}

for player_tuple in players:
    
    player_name = player_tuple[0] 
    
    champion_count[player_name] = 0

for i in range(num_runs):
    winner = run_tournament(players, play, best_of=50, first_player_distribution=0.5 )
    champion_count[winner[0]] += 1

best_agent = max(champion_count.items(), key=lambda x: x[1])

print(f"El mejor agente es {best_agent[0]} con {best_agent[1]} torneos ganados de {num_runs}")


# Distribución de nodos visitados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

agent_name = "Tercera versión"
MCTS = None
for nombre_agente, clase_policy_actual in players:
    if nombre_agente == agent_name:
        MCTS = clase_policy_actual
        break

m = MCTS()
m.mount()

state = ConnectState()

metrics_list = []
move_idx = 0

while not state.is_final():
    current_player = state.player  
    action = m.act(state.board)
    
    metrics = m.last_metrics.copy()
    metrics['selected_action'] = action
    metrics['player'] = current_player
    metrics['move'] = move_idx
    metrics_list.append(metrics)
    
    state = state.transition(action)
    move_idx += 1

df = pd.DataFrame(metrics_list)

player_colors = {-1: 'red', 1: 'yellow'}

plt.figure(figsize=(11,5))
sns.lineplot(data=df, x='move', y='nodes_expanded', hue='player', marker='o', palette=player_colors)
plt.title('Nodos expandidos por movimiento (separado por jugador)')
plt.xlabel('Movimiento')
plt.ylabel('Nodos expandidos')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(11,5))
sns.lineplot(data=df, x='move', y='time_elapsed', hue='player', marker='o', palette=player_colors)
plt.title('Tiempo por movimiento (separado por jugador)')
plt.xlabel('Movimiento')
plt.ylabel('Tiempo (s)')
plt.grid(True)
plt.tight_layout()
plt.show()

